# ArUco Detection Viewer

> A visualization tool for exploring ArUco marker detection results. This application loads video frames and their corresponding detection results, allowing users to inspect detected markers, their positions, orientations, and track detection quality across frames.

In [18]:
import ipywidgets as w
from voilab.applications import aruco_detection_viewer
from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt
import io
import base64

# BUG: the existing implementation is missing aruco tag detection using input intrinsic parameters

# Auto-discover session directories
def find_videos(root):
    """Find session directories (those containing numbers in the name)."""
    session_dirs = []
    return sorted(list(str(item.parent) for item in Path(root).rglob("demo_*/raw_video.mp4")))

# Demo dropdown widget
demo_dropdown = w.Dropdown(
    options=[],
    description="Demo:",
    style={'description_width': 'initial'}
)

# Manual session directory entry
manual_session_widget = w.Text(
    value="",
    placeholder="Path",
    description="Directory:",
    layout=w.Layout(width="400px")
)

# Size controls
fig_width = w.IntSlider(value=4, min=4, max=15, step=1, description="Width (in):")
fig_height = w.IntSlider(value=5, min=3, max=12, step=1, description="Height (in):")
dpi = w.IntSlider(value=100, min=50, max=200, step=10, description="DPI:")

# Parameter file widgets with file pickers
def pick_file(b):
    import tkinter as tk
    from tkinter import filedialog
    root = tk.Tk()
    root.withdraw()
    
    if b.description == "Browse Camera":
        filename = filedialog.askopenfilename(
            title="Select Camera Intrinsics JSON",
            filetypes=[("JSON files", "*.json"), ("All files", "*.*")]
        )
        if filename:
            camera_intrinsics_widget.value = filename
    elif b.description == "Browse ArUco":
        filename = filedialog.askopenfilename(
            title="Select ArUco Config YAML",
            filetypes=[("YAML files", "*.yaml"), ("YAML files", "*.yml"), ("All files", "*.*")]
        )
        if filename:
            aruco_config_widget.value = filename

camera_intrinsics_widget = w.Text(
    value="",
    placeholder="Path to camera intrinsics JSON file",
    description="Camera Intrinsics:",
    layout=w.Layout(width="400px")
)

camera_browse_button = w.Button(description="Browse Camera", button_style="info")

aruco_config_widget = w.Text(
    value="",
    placeholder="Path to ArUco config YAML file",
    description="ArUco Config:",
    layout=w.Layout(width="400px")
)

aruco_browse_button = w.Button(description="Browse ArUco", button_style="info")

# Connect browse buttons
camera_browse_button.on_click(pick_file)
aruco_browse_button.on_click(pick_file)

# Quick load buttons for common configurations
quick_load_section = w.HTML("<h3>Quick Load Configurations</h3>")

def load_gopro13_config(_):
    base_path = Path("packages/umi/defaults/calibration")
    camera_intrinsics_widget.value = str(base_path / "gopro_intrinsics_2_7k.json")
    aruco_config_widget.value = str(base_path / "aruco_config.yaml")

def load_session_dir(_):
    dirs=find_videos(manual_session_widget.value)
    demo_dropdown.options = dirs

gopro13_button = w.Button(description="Load GoPro 13 Config", button_style="warning")
gopro13_button.on_click(load_gopro13_config)
load_session_dir_button = w.Button(description="Load session dir", button_style="warning")
load_session_dir_button.on_click(load_session_dir)

# Run button
run_button = w.Button(description="Load Viewer", button_style="success")
output = w.Output()

# Form layout
form = w.VBox([
    w.HBox([manual_session_widget, load_session_dir_button]),
    demo_dropdown,
    w.HBox([fig_width, fig_height, dpi]),
    quick_load_section,
    w.HBox([gopro13_button]),
    w.HBox([camera_intrinsics_widget, camera_browse_button]),
    w.HBox([aruco_config_widget, aruco_browse_button]),
    run_button,
    output
])
display(form)


def load_and_show(_):
    with output:
        output.clear_output()
        try:
            if manual_session_widget.value.strip():
                session_dir = manual_session_widget.value.strip()
            else:
                print("❌ Please select a session directory")
                return
                
            session_path = Path(session_dir)
            
            if not session_path.exists():
                print(f"❌ Session directory does not exist: {session_dir}")
                return
                
            if not demo_dropdown.value:
                print("❌ Please select a demo")
                return
                
            # Construct the full demo directory path
            demo_dir = session_path / "demos" / demo_dropdown.value
            
            if not demo_dir.exists():
                print(f"❌ Demo directory does not exist: {demo_dir}")
                return
                
            # Check for required files
            video_path = demo_dir / "raw_video.mp4"
            pkl_path = demo_dir / "tag_detection.pkl"
            
            if not video_path.exists():
                print(f"❌ Video file not found: {video_path}")
                return
                
            if not pkl_path.exists():
                print(f"❌ Detection results not found: {pkl_path}")
                return
            
            print("✅ Loading ArUco Detection Viewer...")
            print(f"📁 Session: {session_dir}")
            print(f"🎬 Demo: {demo_dropdown.value}")
            
            # Show optional parameters
            if camera_intrinsics_widget.value.strip():
                print(f"📷 Camera Intrinsics: {camera_intrinsics_widget.value}")
            if aruco_config_widget.value.strip():
                print(f"🏷️  ArUco Config: {aruco_config_widget.value}")
            
            # Load the viewer
            aruco_detection_viewer.show(
                str(demo_dir),
                figsize=(fig_width.value, fig_height.value),
                dpi=dpi.value
            )
            
        except Exception as e:
            print(f"❌ Error: {e}")
            print("\n📋 Troubleshooting:")
            print("- Ensure the session directory exists (e.g., 0826_basement_test)")
            print("- Ensure the session directory contains a 'demos' subdirectory")
            print("- Ensure the demo directory contains both raw_video.mp4 and tag_detection.pkl")
            print("- Check that camera intrinsics file is valid JSON if provided")
            print("- Check that ArUco config file is valid YAML if provided")
            print("- Make sure PyYAML is installed: pip install PyYAML")


run_button.on_click(load_and_show)